In [ ]:
pip install olefile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 2.5 MB/s eta 0:00:00


In [ ]:
pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.6 MB/s eta 0:00:00


In [ ]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00


In [ ]:
pip install gethwp

In [35]:
# pip install olefile

import olefile
import zlib
import struct
import re
from docx import Document
from PyPDF2 import PdfReader
import gethwp

import re

def clean_extracted_text(raw_text):
    """
    Extracted HWPX 텍스트를 정제하는 함수.

    단계별로 불필요한 패턴과 문자를 제거하고, 공백과 문장 부호를 정리합니다.

    Args:
        raw_text (str): 정제할 원본 텍스트.

    Returns:
        str: 정제된 텍스트.
    """

    # 0. '^숫자'로 시작하는 모든 라인 제거
    # 예: '^1.\n^2.\n^3)\n' 등과 같은 패턴을 제거
    text = re.sub(r'^\^\(?\d+[.\)]*\n?', '', raw_text, flags=re.MULTILINE)

    # 1. '7 8 ' 패턴 제거 (남아있을 경우)
    text = re.sub(r'\b7\s*8\b\s*', '', text)

    # 2. 'IAA'와 같은 특정 단어 제거 (대소문자 구분 없이)
    text = re.sub(r'\bIAA\b', '', text, flags=re.IGNORECASE)

    # 3. Base64와 유사한 인코딩된 데이터 제거 (10자 이상)
    # 숫자만으로 이루어진 긴 문자열은 제거하지 않도록 패턴 수정
    text = re.sub(r'\b(?=.*[A-Za-z])[A-Za-z0-9+/=]{10,}\b', '', text)

    # 4. 불필요한 기호 패턴 제거
    # 예: '^1.', '^2)', '(^5)', '(^6)' 등 제거
    text = re.sub(r'\^\d+[.)]', '', text)
    text = re.sub(r'\(\^\d+\)', '', text)

    # 5. 허용된 문자 외의 문자 제거
    # 한글, 영문, 숫자, 공백, '.', ',', '!', '?', '-' 제외한 모든 문자 제거
    text = re.sub(r'[^\uAC00-\uD7A3a-zA-Z0-9\s.,!?-]', ' ', text)

    # 6. 여러 공백을 단일 공백으로 축소
    text = re.sub(r'\s+', ' ', text)

    # 7. 문장 부호 앞뒤의 공백 제거
    # 예: '안녕하세요 . ' -> '안녕하세요.'
    text = re.sub(r'\s*([.,!?])\s*', r'\1', text)

    # 8. 문장 부호 뒤에 단일 공백 추가 (문장 끝은 제외)
    # 예: '안녕하세요.저는' -> '안녕하세요. 저는'
    text = re.sub(r'([.,!?])(?!$)', r'\1 ', text)

    # 9. 앞뒤 공백 제거
    text = text.strip()

    return text

def get_hwp_text(filename):
    f = olefile.OleFileIO(filename)
    dirs = f.listdir()

    if ["FileHeader"] not in dirs or ["\x05HwpSummaryInformation"] not in dirs:
        raise Exception("Not Valid HWP.")

    header = f.openstream("FileHeader")
    header_data = header.read()
    is_compressed = (header_data[36] & 1) == 1

    nums = []
    for d in dirs:
        if d[0] == "BodyText":
            nums.append(int(d[1][len("Section"):]))
    sections = ["BodyText/Section" + str(x) for x in sorted(nums)]

    text = ""
    for section in sections:
        bodytext = f.openstream(section)
        data = bodytext.read()
        try:
            unpacked_data = zlib.decompress(data, -15) if is_compressed else data
        except zlib.error as e:
            print(f"Decompression error: {e}")
            continue

        section_text = ""
        i = 0
        size = len(unpacked_data)
        while i < size:
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type == 67:
                rec_data = unpacked_data[i + 4:i + 4 + rec_len]
                try:
                    decoded_text = rec_data.decode('utf-16-le', errors='ignore')

                    # 한글, 숫자, 영어, 공백만 유지
                    cleaned_text = re.sub(r'[^\uAC00-\uD7A3a-zA-Z0-9\s]', '', decoded_text)
                    section_text += cleaned_text + "\n"
                except UnicodeDecodeError as e:
                    print(f"Decoding error at position {i}: {e}")

            i += 4 + rec_len

        text += section_text
        text += "\n"

    return text

def get_hwpx_text(filename):
    """
    HWPX 파일에서 텍스트를 추출하고 정제하는 함수.

    Args:
        filename (str): HWPX 파일 경로.

    Returns:
        str: 정제된 텍스트 또는 None (추출 실패 시).
    """
    try:
        hwp = gethwp.read_hwpx(filename)
        cleaned_text = clean_extracted_text(hwp)
        return cleaned_text
    except Exception as e:
        print(f"HWPX 파일 읽기 오류: {e}")
        return None

def get_docx_text(filename):
    try:
        doc = Document(filename)
        text = '\n'.join([paragraph.text for paragraph in doc.paragraphs])
        return text
    except Exception as e:
        print("Error reading DOCX file:", e)
        return None

def get_txt_text(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            text = file.read()
        return text
    except Exception as e:
        print("Error reading TXT file:", e)
        return None

def get_pdf_text(filename):
    """
    PDF 파일에서 텍스트를 추출합니다.

    Args:
        filename (str): PDF 파일 경로.

    Returns:
        str: 추출된 텍스트 또는 None (추출 실패 시).
    """
    try:
        reader = PdfReader(filename)
        text = ""
        for page_num, page in enumerate(reader.pages):
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
        return text
    except Exception as e:
        print("Error reading PDF file:", e)
        return None

def extract_text(file_path):
    extension = file_path.split('.')[-1].lower()
    if extension == 'hwp':
        try:
            return get_hwp_text(file_path)
        except Exception as e:
            print(f"Error processing HWP file: {e}")
            return None
    elif extension == 'docx':
        return get_docx_text(file_path)
    elif extension == 'txt':
        return get_txt_text(file_path)
    elif extension == 'pdf':
        return get_pdf_text(file_path)
    elif extension == 'hwpx':
        return get_hwpx_text(file_path)
    else:
        print("Unsupported file type.")
        return None

file_path = '/content/테스트 대본1.txt'  # hwp. docx, txt, pdf, hwpx
text = extract_text(file_path)
print(text)

발표 시작하겠습니다. 안녕하세요. 저는 이번에 인공지능과 파이썬으로 금융 데이터 분석해보기
with 자연어처리로 발표하게 될 김대현이라고 합니다. 일단 제 소개를 간단하게 드리자면, 저는
지금 모듈 연구소라는 곳에서 비전 랭귀지 쪽으로 모델 연구를 하고 있고요. 또한 카카오테크
부트캠프에서 생선형 인공지능 과정을 소강하고 있습니다. 최근에 고파콘이라고 고언어 관련해서
열리는 큰 행사가 있는데 거기 준비위원회로 참가를 했고 이제 또 JRC 라는 로봇 회사에서
로보틱스 엔지니어로 근무한 경력이 있습니다.

일단은 크게 4 가지 주제를 다뤄볼 거에요. 이제 TF-IDF, N-GRAM, COVERT, 다이노싱 메소드,
포지티브 인덱스 리얼리티, 즉 이제 신뢰지수에 대해서 한번 얘기를 해보려고 합니다. 일단은 이
프로젝트를 먼저 시작하게 된 계기가 바로 이거였어요. 일반적으로 우리가 주식을 할 때 처음에
화면을 보면 어떠한 걸 봐야 될지 모르겠는가 에요. 그래서 복잡한 리서치 화면 및 뉴스 커뮤니티에
통한 투자 결정에 어려움이 존재해서 이 프로젝트를 진행하게 되었고 또한 이제 일단은 한국
투자증권 기준으로 보면 카테고리가 일단은 종목이 2400 개가 있어요. 이것을 10 개의 섹터로만
구분해서 의사결정에 대해서 어려운 점이 있었습니다. 그래서 이 프로젝트를 진행하게 되었고
요약해드리자면 이제 우리가 이걸 왜 하게 되었는지 중요 정보에 쉽게 접근하고 이제 정확한 투자
동향을 파악하면서 이제 유망한 분야에 탐색하고 또한 이제 여론과 어느정도 상관관계 분석이
있는지 그거를 파악하면서 개발해 보려고 했습니다.

일단은 개발 설계 및 연구방법에 대해서 설명을 드리면 일단은 모델을 선정이랑 이제 어떻게
프로젝트를 진행할지 기획을 했고요. 건담의 이제 데이터 수집 및 전처리, 부위 체계 세부나
키워드 감성 분석, 적용 데이터 분석, 데이터 시각화 이렇게 다섯가지로 구성을 했습니다. 일단
일반적으로 인공지능 관련한 프로젝트를 시작할 때 이제 데이터 수집을 해야 되잖아요. 그래서
일반

In [ ]:
import re

def clean_extracted_text(raw_text):
    """
    Extracted HWPX 텍스트를 정제하는 함수.

    단계별로 불필요한 패턴과 문자를 제거하고, 공백과 문장 부호를 정리합니다.

    Args:
        raw_text (str): 정제할 원본 텍스트.

    Returns:
        str: 정제된 텍스트.
    """

    # 0. '^숫자'로 시작하는 모든 라인 제거
    # 예: '^1.\n^2.\n^3)\n' 등과 같은 패턴을 제거
    text = re.sub(r'^\^\(?\d+[.\)]*\n?', '', raw_text, flags=re.MULTILINE)

    # 1. '7 8 ' 패턴 제거 (남아있을 경우)
    text = re.sub(r'\b7\s*8\b\s*', '', text)

    # 2. 'IAA'와 같은 특정 단어 제거 (대소문자 구분 없이)
    text = re.sub(r'\bIAA\b', '', text, flags=re.IGNORECASE)

    # 3. Base64와 유사한 인코딩된 데이터 제거 (10자 이상)
    # 숫자만으로 이루어진 긴 문자열은 제거하지 않도록 패턴 수정
    text = re.sub(r'\b(?=.*[A-Za-z])[A-Za-z0-9+/=]{10,}\b', '', text)

    # 4. 불필요한 기호 패턴 제거
    # 예: '^1.', '^2)', '(^5)', '(^6)' 등 제거
    text = re.sub(r'\^\d+[.)]', '', text)
    text = re.sub(r'\(\^\d+\)', '', text)

    # 5. 허용된 문자 외의 문자 제거
    # 한글, 영문, 숫자, 공백, '.', ',', '!', '?', '-' 제외한 모든 문자 제거
    text = re.sub(r'[^\uAC00-\uD7A3a-zA-Z0-9\s.,!?-]', ' ', text)

    # 6. 여러 공백을 단일 공백으로 축소
    text = re.sub(r'\s+', ' ', text)

    # 7. 문장 부호 앞뒤의 공백 제거
    # 예: '안녕하세요 . ' -> '안녕하세요.'
    text = re.sub(r'\s*([.,!?])\s*', r'\1', text)

    # 8. 문장 부호 뒤에 단일 공백 추가 (문장 끝은 제외)
    # 예: '안녕하세요.저는' -> '안녕하세요. 저는'
    text = re.sub(r'([.,!?])(?!$)', r'\1 ', text)

    # 9. 앞뒤 공백 제거
    text = text.strip()

    return text

In [ ]:
import gethwp

try:
    hwp = gethwp.read_hwpx('/content/테스트 대본1.hwpx')
    cleaned_text = clean_extracted_text(hwp)
    print(cleaned_text)
except Exception as e:
    print(f"Error reading HWPX file: {e}")

발표 시작하겠습니다. 안녕하세요. 저는 이번에 인공지능과 파이썬으로 금융 데이터 분석해보기 with 자연어처리로 발표하게 될 김대현이라고 합니다. 일단 제 소개를 간단하게 드리자면, 저는 지금 모듈 연구소라는 곳에서 비전 랭귀지 쪽으로 모델 연구를 하고 있고요. 또한 카카오테크 부트캠프에서 생선형 인공지능 과정을 소강하고 있습니다. 최근에 고파콘이라고 고언어 관련해서 열리는 큰 행사가 있는데 거기 준비위원회로 참가를 했고 이제 또 JRC 라는 로봇 회사에서 로보틱스 엔지니어로 근무한 경력이 있습니다. 일단은 크게 4 가지 주제를 다뤄볼 거에요. 이제 TF-IDF, N-GRAM, COVERT, 다이노싱 메소드, 포지티브 인덱스 리얼리티, 즉 이제 신뢰지수에 대해서 한번 얘기를 해보려고 합니다. 일단은 이 프로젝트를 먼저 시작하게 된 계기가 바로 이거였어요. 일반적으로 우리가 주식을 할 때 처음에 화면을 보면 어떠한 걸 봐야 될지 모르겠는가 에요. 그래서 복잡한 리서치 화면 및 뉴스 커뮤니티에 통한 투자 결정에 어려움이 존재해서 이 프로젝트를 진행하게 되었고 또한 이제 일단은 한국 투자증권 기준으로 보면 카테고리가 일단은 종목이 2400 개가 있어요. 이것을 10 개의 섹터로만 구분해서 의사결정에 대해서 어려운 점이 있었습니다. 그래서 이 프로젝트를 진행하게 되었고 요약해드리자면 이제 우리가 이걸 왜 하게 되었는지 중요 정보에 쉽게 접근하고 이제 정확한 투자 동향을 파악하면서 이제 유망한 분야에 탐색하고 또한 이제 여론과 어느정도 상관관계 분석이 있는지 그거를 파악하면서 개발해 보려고 했습니다. 일단은 개발 설계 및 연구방법에 대해서 설명을 드리면 일단은 모델을 선정이랑 이제 어떻게 프로젝트를 진행할지 기획을 했고요. 건담의 이제 데이터 수집 및 전처리, 부위 체계 세부나 키워드 감성 분석, 적용 데이터 분석, 데이터 시각화 이렇게 다섯가지로 구성을 했습니다. 일단 일반적으로 인공지능 관련한 프로젝트를 시작할 때 이제 데이터 수집을 해야 되잖아요. 그래서 일반적으